In [1]:
def fec_n_meses(fecha, n_meses):
    
    from dateutil.relativedelta import relativedelta
    
    fecha = datetime.datetime.strptime(fecha, '%Y%m%d')
    
    nueva_fecha = fecha-relativedelta(months=-n_meses)
    
    return datetime.datetime.strftime(nueva_fecha, '%Y%m%d')

def n_months_betweeen(m_from, m_to):
    import datetime
    from dateutil.relativedelta import relativedelta
    # n_mon = -1* (relativedelta(m_from,m_to).years*12 + relativedelta(m_from,m_to).months) 
    n_mon = -1* (relativedelta(m_from + datetime.timedelta(days = 1),m_to + datetime.timedelta(days = 1)).years*12 
                 + relativedelta(m_from + datetime.timedelta(days = 1),m_to + datetime.timedelta(days = 1)).months)
    return n_mon

def last_day_of_month(any_day):
    any_day = datetime.datetime.strptime(any_day, '%Y%m%d')
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return datetime.datetime.strftime(next_month - datetime.timedelta(days=next_month.day), '%Y%m%d')

def partition_list(p_ini, p_fin, quotes = True):
    import datetime
    from dateutil.relativedelta import relativedelta

    ## Generar lista de fechas (particiones) a partir de inicio y fin
    m_from = datetime.datetime.strptime(p_ini, '%Y%m%d')
    m_to = datetime.datetime.strptime(p_fin, '%Y%m%d')

    # n_mon = -1* (relativedelta(m_from,m_to).years*12 + relativedelta(m_from,m_to).months) 
    n_mon = n_months_betweeen(m_from, m_to)

    aux_step = -1 if n_mon<0 else 1  ## Permitir listas en orden descendente
    
    my_partition_list = []
    for x in range(0, n_mon+aux_step, aux_step):
        my_partition_list.append(m_from + datetime.timedelta(days = 1) + relativedelta(months = x) - datetime.timedelta(days = 1))
    my_partition_list = [fecha.strftime('%Y%m%d') for fecha in my_partition_list]
    if quotes:
        my_partition_list = ["'"+fecha+"'" for fecha in my_partition_list]
    
    return my_partition_list

def get_partitions(table, partition_var = 'partition_id'):
    return sorted(sq.sql(f"""select distinct {partition_var} from {table}""").toPandas()[partition_var].tolist())

def udf_decorator(type_):
    """
    @Args:
        - type_: type of the resultant udf
    @Computes:
        - a decorator of udfs
    """
    
    def decorator(f):
        return F.udf(f, type_)
    return decorator

anyo_bisiesto = (lambda x: True if x%4==0 and (x%100!=0 or x%400==0) else False)

In [2]:
def crear_tabla (schema, tabla, comentario, var_particionado, **variables):
    
    j = 0
    
    var_particionado = var_particionado + " string"

    for i in list(variables.keys()):
        
        if j == 0:
            
            list_var = i + " " + variables[i]
        
        else:
            
            list_var = list_var + ", " + i + " " + variables[i]
            
        j = j+1
    
    query_creacion = f'''create table {schema}.{tabla} ({list_var}) partitioned by ({var_particionado}) stored as parquet
    TBLPROPERTIES ('comment' = '{comentario}')'''
    
    sq.sql(query_creacion)
    
    print(f"La tabla {tabla} ha sido creada")

In [3]:
def generar_particion (schema, nombre_tabla, p_inicio, p_fin, var_particionado, query_str):
    
    for p_ini in partition_list(p_inicio, p_fin,False):
    
        particion = last_day_of_month(p_ini)

        if sq.sql(f'''select count(*) as N from {schema}.{nombre_tabla} where {var_particionado} = {particion}''')\
        .toPandas()['N'].tolist()[0] > 0:

            texto = "La partición {0} de {1} ya existe".format(particion, nombre_tabla)

            print(texto)

        else:
            
            texto = 'Ejecutando la particion {0} de {1}'.format(particion, nombre_tabla)

            print(texto)
            
            sq.sql(query_str.format(schema, nombre_tabla, p_ini, particion))

            texto = "Ejecutada la particion {0} de {1}".format(particion, nombre_tabla)
            
            print(texto)